# Introducing PlesioGeostroPy: Quickstart

PlesioGeostroPy is a python realization of the plesio-gestrophy model. Plesio-Geostrophy model (see [Jackson and Maffei, 2020](https://doi.org/10.1098/rspa.2020.0513)), or PG model, is a 2-D reduced model for 3-D MHD in rapidly rotating frame, based on near-geostrophic flows in an axisymmetric cavity. The model uses a plesio-geostrophic ansatz, where the velocity field is completely described by a scalar stream function, but remains solenoidal and satisfies the non-penetration boundary condition. In diffusionless case, the MHD equations are shown to exactly reduce to a set of equations involving only the stream function and the axially integrated magnetic moments (except for the boundary terms), which are all 2-D variables.

It should be noted that the current development involves only a full sphere geometry. However, the method would apply to general axisymmetric cavities without an inner core.

## Intro

### Why PlesioGeostroPy?

Indeed, we already have Daria's Wolfram Mathematica notebook, which also heavily inspired the initial development of this small package. The decision to develop a python realization is motivated by the following observations:
- Despite the completeness of Daria's notebook, many notations are cryptic and not easy to apprehend.
    - ***Problem***: The moments have been replaced with notations `A` to `H`, whereas the components of magnetic fields are replaced with notations `a` to `c`. For instance, field $\overline{M_{ss}}=\overline{B_s^2}$ is `A`, $\overline{M_{s\phi}} = \overline{B_sB_\phi}$ is `C` (or `Csp` for the perturbation field). The equations also use a confusing numbering system. Not only do we have `eq1` to `eq11`, but also `eq73`, `eq74`, `eq76` and `eq80`. These equations seem to correspond to the not-yet-linearized version of the induction equations. None of these are physically meaningful notations, and I even have jotted down a mapping table just to decipher the Mathematica code. 
    - ***Cause***: non-physical namings.
    - ***Response***: Given the former two observations, a rewrite of the code is desired.
- The original mathematica notebook lacks flexibility when it comes to changing expansions, plottings, numerical computations, etc.
    - ***Problem***: Changing expansions would require switching on and off code blocks, or creating new notebooks. Plotting (according to Stefano) has always been confusing in Mathematica. In addition to that, many of the Mathematica functions, e.g. special functions, etc. do not vectorize well, and would not be efficient in numerics.
    - ***Cause***: many of the problems can be attributed to the fact that Mathematica is a domain-specific language (DSL). The inflexibility is due to the fact that Mathematica is mostly a functional language for symbolic manipulation, whereas the support of object-oriented programming is very limited (except for [a OOP repo](https://library.wolfram.com/infocenter/Articles/3243/) from 30 years ago, the links from which have mostly expired). The numerics part is not comparable to numerical libaries (e.g. `numpy`, `scipy`), let alone numerics DSL such as MATLAB. In addition, it is also complicated to use external libraries for faster numerical computation.
    - ***Response***: Program the model in a language that is a general-purpose language (GPL), which allows abstract design of the code and at the same time allows efficient numerical algorithms. Designed initially to be a scripting language or glue language, and developing later into a GPL with numerous external packages, Python is a good choice. The packages such as `sympy` and `numpy` allow good integration between the symbolic and the numeric sides of the picture, while `matplotlib` and other visualization packages provide an additional selling point. It is also much easier to either directly call or export intermediate variables to external numerical libraries.
- Mathematica is closed source, while python itself as well as many of the packages are open source.

Of course, switching from Daria's Mathematica notebook to PlesioGeostroPy implementation comes with a price.
- Symbolic computation may be more robust in Mathematica than in Python. This is statement recycled from some Mathematica users. This might be true for solving differential equations or other equations symbolically, however this is not relevant for PG model. In simple algebras, simplifications and substitutions, Mathematica is just as "capricious" as `Sympy`, the behaviour of neither would coincide exactly as what a human would do during derivations. On top of that, there is no knowing what exactly went wrong in Mathematica, whereas in `Sympy`, the user can peek under the hood as the code is open-source.
- Arbitrary precision computation is slower in vanilla-version `SymPy`, however slightly. This may be a real issue, but may also be circumvented by using multi-precision evaluation libraries, but it would be much more desirable if computing within the double precision is sufficient.

### Prerequisites

The package **PlesioGeostroPy** is written in python, and dependent on the following packages.

```python
# Core packages
python=3.8.13
numpy=1.22.3
scipy=1.8.0
sympy=1.11.1
h5py=3.3.0

# Visualization and presentation
pandas=1.4.2
matplotlib=3.5.2
jupyterlab=3.4.0

# Documentation and testing
sphinx=5.0.2
pytest=7.4.2
perfplot=0.10.2
line_profiler=4.1.1
```

In addition, in order to harness the power of the pretty printing interface, it is highly recommended that the interactive programming is done using [web-based Project Jupyter tools](https://jupyter.org/), such as JupyterLab, Jupyter notebook, or IPython consoles launched in Jupyter services. These tools enable LaTeX rendering, which is a significantly improvement compared to Unicode or ASCII printing methods.

---

## Getting started with SymPy

`Sympy`, or "Symbolic Python", is a Python library for symbolic mathematics. A very brief demo is covered here. For a longer tutorial, see [the official website](https://docs.sympy.org/latest/tutorials/intro-tutorial/index.html#intro-tutorial).

As in using other Python libraries, to import utilities from the package one only needs to write

In [1]:
from sympy import *

### Symbols, functions, expressions

As in most programming languages, one needs to **define** a variable before using it, and symbolic symbols are no exceptions in `SymPy`. This might be the only possible and reasonable thing to do for anyone with a background in C/C++, Java, C#, Fortran, Python, Julia or even MATLAB. The only scenarios, where I know the variables are readily interpreted as symbols even before definition are Mathematica and Maple.

To define a symbolic variable, which will have the name/expression $\psi_1^{m}$, we can write the following code

In [2]:
psi_1m = Symbol(r"\hat{\psi}_1^m")
psi_1m

\hat{\psi}_1^m

This example already shows that you can use just about any LaTeX expressions as the symbol of the variable.

A symbol is just an independent variable, and has no known dependency on any other variables:

In [3]:
x, y, z = symbols("x, y, z")

diff_psi_1m_x = diff(psi_1m, x)
diff_psi_1m_x

0

If a dependency on some variable is needed for some symbol, one needs to define a **function**:

In [4]:
psi = Function(r"\psi")(x, y, z)

diff_psi_x = diff(psi, (x, 2), (y, 1), (z, 0))
diff_psi_x

Derivative(\psi(x, y, z), (x, 2), y)

Adding, substracting, multiplying or dividing any symbols or functions will generate an **expression** (`Expr`):

In [5]:
psi_expr = psi_1m**2*x + psi_1m*y + z
psi_expr

\hat{\psi}_1^m**2*x + \hat{\psi}_1^m*y + z

An expression can once again be added, substracted, multiplied, and differentiated etc. as well. To construct a derivative object without evaluation, use `Derivative`:

In [6]:
dpsi_expr_dx = Derivative(psi_expr, x)
dpsi_expr_dx

Derivative(\hat{\psi}_1^m**2*x + \hat{\psi}_1^m*y + z, x)

One can evaluate this by either invoking the method `.doit()`, or directly using `diff` method:

In [7]:
display(dpsi_expr_dx.doit())
display(diff(psi_expr, x))

\hat{\psi}_1^m**2

\hat{\psi}_1^m**2

In `sympy`, there is a clear hierarchy of classes. `Expr` is pretty much one of the most basic base classes. This means a `Function` is an `Expr`, a `Symbol` is an `Expr`, and a `jacobi` polynomial is also an `Expr`. The results of algebraic manipulations, which may be `Add`, `Mul`, are all `Expr`s. For readers who do not know object-oriented programming, by `A` *is* `Expr`, what I really mean is that `A` is a child class of, or inherit from, `Expr`.

### Substitutions, simplifications

Substitution in Sympy is realized by the `Expr.subs()` method. A typical way to use the substitution method is to pass a dictionary (a key-value "map" in C++ languages) into the method.

This is much more human readable than the `/.` operator in Mathematica.

The following example shows how the $\psi$ function is replaced with a Jacobi polynomial of order $n$.

In [8]:
alpha = Symbol(r"\alpha")
beta = Symbol(r"\beta")
n = Symbol('n', integer=True)

diff_jacobi = diff_psi_x.subs({psi: jacobi(n, alpha, beta, x)})
diff_jacobi

Derivative(jacobi(n, \alpha, \beta, x), (x, 2), y)

Substituting a variable whose derivative is involved would result in delayed substitution:

In [9]:
diff_val = diff_psi_x.subs({x: 1, y: 1})
diff_val

Subs(Derivative(\psi(x, y, z), (x, 2), y), (y, x), (1, 1))

which can only be evaluated when the explicit form is calculated:

In [10]:
diff_val = diff_val.subs({psi: x**3*y**2}).doit()
diff_val

12

Simplification is probably the most counterintuitive part of all symbolic computation libraries, simply due to the fact that the computer does not know for sure what the human programmer wants.

In many cases, when we are doing simplifications, we also do it by trial and error, and we don't even know *a priori* what form we want to end up in.

We can make a dummy example here,

In [11]:
diff_simp = diff_psi_x.subs({psi: x**3*y**2 + 1/x/y}).simplify()
diff_simp

2*(7*x*y - (x**4*y**3 + 1)/(x**3*y**2))

Apparently, it is much reasonable to rewrite it as
$$
2\left(6xy - \frac{1}{x^3 y^2}\right)
$$, this can still be achieved using `expand()`

In [12]:
diff_simp.expand()

12*x*y - 2/(x**3*y**2)

For more on simplifying expressions such as rational expressions, special functions, etc., see [Simplifiations in SymPy](https://docs.sympy.org/latest/tutorials/intro-tutorial/simplification.html).

---

## Getting started with PlesioGeostroPy

Now we have come to the fun part: the actual utilities concerning the PG model.

The PG equations, especially the induction equations, are of such a complex form that a sane human being is usually not overly fond of writing the expression down very often.
PlesioGeostroPy does just that for you, by wrapping the governing equations in much human-readable structures.

In this section, we shall look at the symbolic part of this package. We will see how to inspect equations, transformations and their linearized versions.

### Core variables

The core variables that are concerned in the model are constructed in `pg_model.core` module.
These first include the coordinates $s$, $\phi$, $z$; Cartesian coordinates $x$, $y$; as well as spherical coordinates $r$, $\theta$.

In [13]:
import os, sys
root_dir = "."

# The following 2 lines are a hack to import from parent directory
# If the notebook is run in the root directory, comment out these 2 lines
sys.path.append(os.path.dirname(os.getcwd()))
root_dir = ".."

from sympy import *
from pg_utils.pg_model import core
from pg_utils.pg_model.core import s, p, z, t

print("Cylindrical oordinates:")
display(*[core.cyl[i_comp] for i_comp in range(3)])

Cylindrical oordinates:


s

\phi

z

Next, we have the 3-D vector fields. These include
- magnetic field: in cylindrical coordinates: `B_vec`, background field `B0_vec`; in spherical coordinates `B_sph`, background field `B0_sph`;
- velocity field: in cylindrical coordinates: `U_vec`, background field `U0_vec`; in spherical coordinates `U_sph`, background field `U0_sph`;

we can take a look at one of these variables.

In [14]:
display(*[core.B_vec[i_comp] for i_comp in range(3)])

B_s(s, \phi, z, t)

B_\phi(s, \phi, z, t)

B_z(s, \phi, z, t)

And finally, the PG variables. These are stored in `core.pgvar`, `core.pgvar_bg` (background) and `core.pgvar_ptb` (perturbation). These are of the type

In [15]:
type(core.pgvar)

pg_utils.pg_model.base.CollectionPG

`CollectionPG` is a special class specifically designed for holding PG variables. This class allows accessing fields as attributes or by index, as well as many other useful features. For more detailed explanation of how this interface works, please refer to [Demo_Collections](Demo_Collections.ipynb).

To look at what variables the object holds, we can for instance make the following inquiry

In [16]:
core.pgvar._field_names

['Psi',
 'Mss',
 'Mpp',
 'Msp',
 'Msz',
 'Mpz',
 'zMss',
 'zMpp',
 'zMsp',
 'Bs_e',
 'Bp_e',
 'Bz_e',
 'dBs_dz_e',
 'dBp_dz_e',
 'Br_b',
 'Bs_p',
 'Bp_p',
 'Bz_p',
 'Bs_m',
 'Bp_m',
 'Bz_m']

We see that we have in all 21 variables. The first is the stream function; the second to the eight are magnetic moments; then we have the fields in the equatorial plane (ones with `_e` suffix). Finally, we have the boundary terms.

Note that in standard PG, only `Br_b` = "radial magnetic field at the boundary" is used; however, for many problems such as the eigenvalue problem, we can use `Bs_p` (s-magnetic field at $z=+H$) - `Bz_m` (z-magnetic field at $z=-H$) instead of `Br_b`. As mentioned, all these fields can be accessed just by typing in the attribute name:

In [17]:
display(core.pgvar.Psi, core.pgvar.Bz_m)

\Psi(s, \phi, t)

B_z^-(s, \phi, t)

In addition to this method of accessing fields, the same variable `pgvar` can also be indexed.

In [18]:
display(core.pgvar[0], core.pgvar[-1])

\Psi(s, \phi, t)

B_z^-(s, \phi, t)

And finally, as a sugar frosting, the fields can be traversed by iterating through the variable `pgvar`.

In [19]:
display(*[field_symb for field_symb in core.pgvar])

\Psi(s, \phi, t)

\overline{M_{ss}}(s, \phi, t)

\overline{M_{\phi\phi}}(s, \phi, t)

\overline{M_{s\phi}}(s, \phi, t)

\widetilde{M_{sz}}(s, \phi, t)

...

B_\phi^+(s, \phi, t)

B_z^+(s, \phi, t)

B_s^-(s, \phi, t)

B_\phi^-(s, \phi, t)

B_z^-(s, \phi, t)

The same thing holds for `pgvar_bg` and `pgvar_ptb`. The background fields are denoted with an additional $0$ superscript, and the perturbed fields are denoted with lower-case letters.

In [20]:
display(core.pgvar_bg.Mss, core.pgvar_ptb.Mss)

\overline{M_{ss}}^0(s, \phi)

\overline{m_{ss}}(s, \phi, t)

Last but not least, the `core` module also stores the PG ansatz of the velocity, which can be invoked whenever necessary:

In [21]:
display(*[core.U_pg[i_comp] for i_comp in range(3)])

Derivative(\Psi(s, \phi, t), \phi)/(s*H(s))

-Derivative(\Psi(s, \phi, t), s)/H(s)

z*Derivative(H(s), s)*Derivative(\Psi(s, \phi, t), \phi)/(s*H(s)**2)

### Equations and linearizations

With all the variables defined, we can now introduce the equations. The 15 PG equations have been typed in manually in the `equations.py` module.
Since these equations are constructed when the module is imported, the first import of `equations` module takes some time (~10s or so).

These are similarly stored in a `CollectionPG` object called `equations.eqs_pg`. Let us inspect these equations:

In [22]:
from pg_utils.pg_model import equations as eqs

display(*[eq for eq in eqs.eqs_pg])

Eq((-Derivative(H(s), s)/(2*H(s)**2) + 1/(s*H(s)))*Derivative(\Psi(s, \phi, t), (\phi, 2), t) + Derivative(s*Derivative(\Psi(s, \phi, t), s, t)/H(s), s), -s*((s*Derivative(\overline{f_\phi}(s, \phi, t), s) + \overline{f_\phi}(s, \phi, t))/s - Derivative(\overline{f_s}(s, \phi, t), \phi)/s)/(2*H(s)) + (s*f_{\phi}^e(s, \phi, t)/H(s) + Derivative(\widetilde{f_z}(s, \phi, t), \phi)/(2*H(s)))*Derivative(H(s), s) - 2*Derivative(H(s), s)*Derivative(\Psi(s, \phi, t), \phi)/H(s)**2)

Eq(Derivative(\overline{M_{ss}}(s, \phi, t), t), -(U_s(s, \phi, z, t)*Derivative(\overline{M_{ss}}(s, \phi, t)/H(s), s) + U_\phi(s, \phi, z, t)*Derivative(\overline{M_{ss}}(s, \phi, t)/H(s), \phi)/s)*H(s) + 2*\overline{M_{ss}}(s, \phi, t)*Derivative(U_s(s, \phi, z, t), s) + 2*\overline{M_{s\phi}}(s, \phi, t)*Derivative(U_s(s, \phi, z, t), \phi)/s)

Eq(Derivative(\overline{M_{\phi\phi}}(s, \phi, t), t), 2*s*\overline{M_{s\phi}}(s, \phi, t)*Derivative(U_\phi(s, \phi, z, t)/s, s) - (U_s(s, \phi, z, t)*Derivative(H(s)*\overline{M_{\phi\phi}}(s, \phi, t), s) + U_\phi(s, \phi, z, t)*Derivative(H(s)*\overline{M_{\phi\phi}}(s, \phi, t), \phi)/s)/H(s) - 2*\overline{M_{\phi\phi}}(s, \phi, t)*Derivative(U_s(s, \phi, z, t), s))

Eq(Derivative(\overline{M_{s\phi}}(s, \phi, t), t), s*\overline{M_{ss}}(s, \phi, t)*Derivative(U_\phi(s, \phi, z, t)/s, s) - U_s(s, \phi, z, t)*Derivative(\overline{M_{s\phi}}(s, \phi, t), s) - U_\phi(s, \phi, z, t)*Derivative(\overline{M_{s\phi}}(s, \phi, t), \phi)/s + \overline{M_{\phi\phi}}(s, \phi, t)*Derivative(U_s(s, \phi, z, t), \phi)/s)

Eq(Derivative(\widetilde{M_{sz}}(s, \phi, t), t), (Derivative(U_s(s, \phi, z, t), s) + 2*Derivative(U_z(s, \phi, z, t), z))*\widetilde{M_{sz}}(s, \phi, t) - U_s(s, \phi, z, t)*Derivative(\widetilde{M_{sz}}(s, \phi, t), s) + \widetilde{zM_{ss}}(s, \phi, t)*Derivative(U_s(s, \phi, z, t)*Derivative(H(s), s)/H(s), s) - U_\phi(s, \phi, z, t)*Derivative(\widetilde{M_{sz}}(s, \phi, t), \phi)/s + \widetilde{M_{\phi z}}(s, \phi, t)*Derivative(U_s(s, \phi, z, t), \phi)/s + \widetilde{zM_{s\phi}}(s, \phi, t)*Derivative(H(s), s)*Derivative(U_s(s, \phi, z, t), \phi)/(s*H(s)))

...

Eq(Derivative(B_\phi^+(s, \phi, t), t), B_s^+(s, \phi, t)*Derivative(U_\phi(s, \phi, z, t), s) + B_z^+(s, \phi, t)*Derivative(U_\phi(s, \phi, z, t), z) - U_s(s, \phi, z, t)*Derivative(B_\phi(s, \phi, z, t), s) - U_z(s, \phi, z, t)*Derivative(B_\phi(s, \phi, z, t), z) + (B_\phi^+(s, \phi, t)*U_s(s, \phi, z, t) - B_s^+(s, \phi, t)*U_\phi(s, \phi, z, t))/s + B_\phi^+(s, \phi, t)*Derivative(U_\phi(s, \phi, z, t), \phi)/s - U_\phi(s, \phi, z, t)*Derivative(B_\phi(s, \phi, z, t), \phi)/s)

Eq(Derivative(B_z^+(s, \phi, t), t), B_s^+(s, \phi, t)*Derivative(U_z(s, \phi, z, t), s) + B_z^+(s, \phi, t)*Derivative(U_z(s, \phi, z, t), z) - U_s(s, \phi, z, t)*Derivative(B_z(s, \phi, z, t), s) - U_z(s, \phi, z, t)*Derivative(B_z(s, \phi, z, t), z) + B_\phi^+(s, \phi, t)*Derivative(U_z(s, \phi, z, t), \phi)/s - U_\phi(s, \phi, z, t)*Derivative(B_z(s, \phi, z, t), \phi)/s)

Eq(Derivative(B_s^-(s, \phi, t), t), B_s^-(s, \phi, t)*Derivative(U_s(s, \phi, z, t), s) + B_z^-(s, \phi, t)*Derivative(U_s(s, \phi, z, t), z) - U_s(s, \phi, z, t)*Derivative(B_s(s, \phi, z, t), s) - U_z(s, \phi, z, t)*Derivative(B_s(s, \phi, z, t), z) + B_\phi^-(s, \phi, t)*Derivative(U_s(s, \phi, z, t), \phi)/s - U_\phi(s, \phi, z, t)*Derivative(B_s(s, \phi, z, t), \phi)/s)

Eq(Derivative(B_\phi^-(s, \phi, t), t), B_s^-(s, \phi, t)*Derivative(U_\phi(s, \phi, z, t), s) + B_z^-(s, \phi, t)*Derivative(U_\phi(s, \phi, z, t), z) - U_s(s, \phi, z, t)*Derivative(B_\phi(s, \phi, z, t), s) - U_z(s, \phi, z, t)*Derivative(B_\phi(s, \phi, z, t), z) + (B_\phi^-(s, \phi, t)*U_s(s, \phi, z, t) - B_s^-(s, \phi, t)*U_\phi(s, \phi, z, t))/s + B_\phi^-(s, \phi, t)*Derivative(U_\phi(s, \phi, z, t), \phi)/s - U_\phi(s, \phi, z, t)*Derivative(B_\phi(s, \phi, z, t), \phi)/s)

Eq(Derivative(B_z^-(s, \phi, t), t), B_s^-(s, \phi, t)*Derivative(U_z(s, \phi, z, t), s) + B_z^-(s, \phi, t)*Derivative(U_z(s, \phi, z, t), z) - U_s(s, \phi, z, t)*Derivative(B_z(s, \phi, z, t), s) - U_z(s, \phi, z, t)*Derivative(B_z(s, \phi, z, t), z) + B_\phi^-(s, \phi, t)*Derivative(U_z(s, \phi, z, t), \phi)/s - U_\phi(s, \phi, z, t)*Derivative(B_z(s, \phi, z, t), \phi)/s)

Of course, as other `CollectionPG` objects as `core.pgvar`, the equations in `equations.eqs_pg` can also be accessed either by attribute (e.g. `equations.eqs_pg.Mss`) or by index (e.g. `equations.eqs_pg[1]`).

These equations are then linearized automatically in the same module. The procedure is as follows: each fields are expanded as the background state + $\epsilon\cdot$ a perturbation state. Then, all the terms that are linear in $\epsilon$ are collected to yields the linearized form. This method is borrowed from Daria's Mathematica notebook.

The linearized equations are then stored in `equations.eqs_lin`. For instance, the linearized evolution equation for $\widetilde{zM_{s\phi}}$ takes the form

In [23]:
eqs.eqs_pg_lin.zMsp

Eq(Derivative(\widetilde{zm_{s\phi}}(s, \phi, t), t), -U_s^0(s, \phi, z)*Derivative(\widetilde{zm_{s\phi}}(s, \phi, t), s) + \widetilde{zm_{s\phi}}(s, \phi, t)*Derivative(U_z^0(s, \phi, z), z) + \widetilde{zm_{ss}}(s, \phi, t)*Derivative(U_\phi^0(s, \phi, z), s) - \widetilde{zM_{ss}}^0(s, \phi)*Derivative(\psi(s, \phi, t), (s, 2))/H(s) + \widetilde{zM_{ss}}^0(s, \phi)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), s)/H(s)**2 - U_\phi^0(s, \phi, z)*\widetilde{zm_{ss}}(s, \phi, t)/s - U_\phi^0(s, \phi, z)*Derivative(\widetilde{zm_{s\phi}}(s, \phi, t), \phi)/s + \widetilde{zm_{\phi\phi}}(s, \phi, t)*Derivative(U_s^0(s, \phi, z), \phi)/s + \widetilde{zM_{ss}}^0(s, \phi)*Derivative(\psi(s, \phi, t), s)/(s*H(s)) - Derivative(\psi(s, \phi, t), \phi)*Derivative(\widetilde{zM_{s\phi}}^0(s, \phi), s)/(s*H(s)) + Derivative(\psi(s, \phi, t), s)*Derivative(\widetilde{zM_{s\phi}}^0(s, \phi), \phi)/(s*H(s)) + \widetilde{zM_{s\phi}}^0(s, \phi)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), \phi)/(

- The vorticity equation has been validated against Daria's notebook `quad_malkus_reg_diff.nb`, and has passed the test.
- Induction terms of $\overline{m_{ss}}$, $\overline{m_{\phi\phi}}$, $\overline{m_{s\phi}}$, $\widetilde{m_{sz}}$, $\widetilde{m_{\phi z}}$, $\widetilde{zm_{ss}}$, $z\widetilde{m_{\phi\phi}}$, $z\widetilde{m_{s\phi}}$, as well as $b_{es}$ and $b_{e\phi}$ (i.e. the linearized induction term), have been validated against `quad_malkus_reg_diff.nb` and have passed the test.

In [24]:
eqs.eqs_pg_lin.Bs_p.subs({core.U0_vec.s: 0, core.U0_vec.p: 0, core.U0_vec.z: 0}).doit()

Eq(Derivative(b_s^+(s, \phi, t), t), -z*Derivative(B_s^0(s, \phi, z), z)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), \phi)/(s*H(s)**2) + B_s^{0+}(s, \phi)*Derivative(\psi(s, \phi, t), \phi, s)/(s*H(s)) - B_s^{0+}(s, \phi)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), \phi)/(s*H(s)**2) + Derivative(B_s^0(s, \phi, z), \phi)*Derivative(\psi(s, \phi, t), s)/(s*H(s)) - Derivative(B_s^0(s, \phi, z), s)*Derivative(\psi(s, \phi, t), \phi)/(s*H(s)) + B_\phi^{0+}(s, \phi)*Derivative(\psi(s, \phi, t), (\phi, 2))/(s**2*H(s)) - B_s^{0+}(s, \phi)*Derivative(\psi(s, \phi, t), \phi)/(s**2*H(s)))

- Induction terms of $b_s^\pm$, $b_\phi^\pm$, $b_z^\pm$ has been validated against the expressions in the document [PG_Assim.pdf](./doc/PG_Assim.pdf), which are derived by hand. The validation is successful.
- The validation is performed under zero background velocity.

We note that in the vorticity equation, the placeholders for the forces are used:

In [25]:
eqs.eqs_pg_lin.Psi

Eq(s*Derivative(\psi(s, \phi, t), (s, 2), t)/H(s) - s*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), s, t)/H(s)**2 + Derivative(\psi(s, \phi, t), s, t)/H(s) - Derivative(H(s), s)*Derivative(\psi(s, \phi, t), (\phi, 2), t)/(2*H(s)**2) + Derivative(\psi(s, \phi, t), (\phi, 2), t)/(s*H(s)), s*f_{\phi}^e(s, \phi, t)*Derivative(H(s), s)/H(s) - s*Derivative(\overline{f_\phi}(s, \phi, t), s)/(2*H(s)) - \overline{f_\phi}(s, \phi, t)/(2*H(s)) + Derivative(H(s), s)*Derivative(\widetilde{f_z}(s, \phi, t), \phi)/(2*H(s)) + Derivative(\overline{f_s}(s, \phi, t), \phi)/(2*H(s)) - 2*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), \phi)/H(s)**2)

where $\overline{f_s}$, $\overline{f_\phi}$, $\widetilde{f_z}$ and $f_{e\phi}$ are the vertically evenly integrated $s$-component, $\phi$-component, vertically oddly integrated $z$-component and the equatorial $\phi$-component of arbitrary body force $\mathbf{f}$, respectively.

This allows some flexibility regarding which forces are included in the system. Currently, only the forms of the Lorentz force are compiled in the package, but adding viscous diffusion as well as buoyancy force will be straightforward.

### Precomputed equations

As you probably have noticed, loading the equations from the `pg_utils.pg_model.equations` module is **slooowwww**!

Loading the equations now may take 20-30s (tested on my laptop, AMD R5-4500U).

The reason is that it takes a while to compile the equations, as they are not all written out in explicit forms in the code.
In linearization especially, the program needs to simplify the expressions to a form where the perturbation $\epsilon$ is taken out of all brackets.
Another problem is that so far, compilation and derivation of equations is done sequentially, not in parallel.

While the compilation of equations is not expected to be the bottleneck of the program in the end, it would still be desirable if it can be loaded faster.
A method is to compute the equations and store them in a string that can be quickly loaded.
This has already been done and the outputs have been saved under `out/symbolic/`.
One can use the following code to load a set of equations:

In [26]:
from pg_utils.pg_model import base

with open(os.path.join(root_dir, "out/symbolic/eqs_pg.json"), 'r') as fread:
    pgeqs = base.CollectionPG.load_json(fread, parser=parse_expr)

which cuts back the cost of loading from 20s to 2s. The following four sets are given in the folder:
- `eqs_pg.json`: original PG equations
- `eqs_pg_lin.json`: linearized PG equations
- `eqs_cg.json`: conjugate variable equations
- `eqs_cg_lin.json`: linearized conjugate variable equations

This is now the recommended way to load equations. However, it also means that if the equations in `pg_utils.pg_model.equations` are changed, they would need to be re-compiled and re-saved.

### Forcing and linearization

As mentioned, the forcing in the vorticity equation is only given by placeholder functions. It remains to be defined what forces are involved in the system. This is the task for the `forcing.py` module.

Currently, only the forms of the Lorentz force are compiled in the package. The explicit forms for Lorentz force are stored in `forcing.Ls_sym_expr`, `forcing.Lp_sym_expr`, `forcing.Lz_asym_expr` and `forcing.Le_p_expr`.

In [27]:
from pg_utils.pg_model import forcing

Eq(forcing.Le_p, forcing.Le_p_expr)

Eq(L_{\phi}^e(s, \phi, t), B_{\phi, z}^e(s, \phi, t)*B_{z}^e(s, \phi, t) + B_{s}^e(s, \phi, t)*Derivative(B_{\phi}^e(s, \phi, t), s) + B_{\phi}^e(s, \phi, t)*B_{s}^e(s, \phi, t)/s + B_{\phi}^e(s, \phi, t)*Derivative(B_{\phi}^e(s, \phi, t), \phi)/s)

The linearization of the forces are the same as linearization of equations and is done automatically. The linearized forcing terms are stored in `forcing.Ls_sym_lin`, `forcing.Lp_sym_lin`, `forcing.Lz_asym_lin` and `forcing.Le_p_lin` expressions.

In [ ]:
Eq(forcing.Ls_sym, forcing.Ls_sym_lin)

Eq(\overline{L_s}(s, \phi, t), 2*s*B_s^{0+}(s, \phi)*b_s^+(s, \phi, t)/H(s) + 2*s*B_s^{0-}(s, \phi)*b_s^-(s, \phi, t)/H(s) + B_s^{0+}(s, \phi)*b_z^+(s, \phi, t) - B_s^{0-}(s, \phi)*b_z^-(s, \phi, t) + B_z^{0+}(s, \phi)*b_s^+(s, \phi, t) - B_z^{0-}(s, \phi)*b_s^-(s, \phi, t) + Derivative(\overline{m_{ss}}(s, \phi, t), s) - \overline{m_{\phi\phi}}(s, \phi, t)/s + \overline{m_{ss}}(s, \phi, t)/s + Derivative(\overline{m_{s\phi}}(s, \phi, t), \phi)/s)

Linearized expressions for Lorentz force are validated against Daria's notebook `quad_malkus_reg_diff.nb`.
- $\overline{L_\phi}$, $\widetilde{L_z}$ and $L_\phi(z=0)$ passed the validation;
- The $s$-component of the equatorial Lorentz force in `quad_malkus_reg_diff` has a $\frac{s^2}{H}$ coefficient, while the current derivation (and others) indicate this coefficient should be $\frac{s}{H}$

For a more detailed explanation on the ingredients, see [Variables, equations and forcing](Demo_Variables.ipynb).

## To be continued

Here are more demos to learn about PlesioGeostroPy:
- [The collection interface](Demo_Collections.ipynb)
- [The variables, equations and forcing](Demo_Variables.ipynb)
- [Regularity conditions of tensor components in polar coordinates](Demo_Regularity.ipynb)